In [ ]:
# Load Dataset
import pandas as pd 

file_path = "/Users/Sebastiano/SLE/data/Clinical_MRI.xlsx"
df = pd.read_excel(file_path)

print("N° of patients: {}".format(len(df)))
print("N° of columns: {}".format(df.shape[1]))

pd.set_option('display.max_columns', None)

import os 
# Create the results/ directory if it doesn't exist
if not os.path.exists('results'):
    os.makedirs('results')

# Extract feature matrix X and show 5 random samples
df.head()

N° of patients: 27
N° of columns: 969


# Structure Investigation

In [33]:
# Show size of the dataset
df.shape

(27, 969)

In [34]:
# Count how many times each data type is present in the dataset
pd.value_counts(df.dtypes)

float64    931
int64       30
object       8
dtype: int64

## Structure of non-numerical features

In [ ]:
# Display non-numerical features
df.select_dtypes(exclude="number").head()

In [ ]:
df.describe(exclude="number")

## Structure of numerical features

In [ ]:
# For each numerical feature compute number of unique entries
unique_values = df.select_dtypes(include="number").nunique().sort_values()

# Plot information with y-axis in log-scale
unique_values.plot.bar(logy=True, figsize=(55, 40), title="Unique values");

In [ ]:
df.iloc[:, :11].describe(include="number")

In [ ]:
df.iloc[:,11 :21].describe(include="number")

In [ ]:
df.iloc[:,21 :41].describe(include="number")

In [ ]:
df.iloc[:,41 :61].describe(include="number")

In [ ]:
# Describe

df.iloc[:,6:35].groupby(['NP-SLE']).describe()

# Quality Investigation

## Duplicates

In [4]:
# Check number of duplicates while ignoring the index feature
n_duplicates = df.drop(labels=["Patient"], axis=1).duplicated().sum()
print(f"You seem to have {n_duplicates} duplicates in your database.")

You seem to have 0 duplicates in your database.


In [5]:
#  Extract column names of all features, except 'Accident_Index'
columns_to_consider = df.drop(labels=["Patient"], axis=1).columns

# Drop duplicates based on 'columns_to_consider'
df_X = df.drop_duplicates(subset=columns_to_consider)
df_X.shape

(27, 969)

## Missing Values

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
plt.imshow(df_X.isna(), aspect="auto", interpolation="nearest", cmap="gray")
plt.xlabel("Column Number")
plt.ylabel("Sample Number");

In [ ]:
# Calculate the percentage of missing values in each column
missing_perc = (df_X.isnull().sum() / len(df_X)) * 100

# Calculate the percentage of unique values in each column
unique_perc = (df_X.nunique() / len(df_X)) * 100

# Calculate the skewness of each column
skewness = df_X.skew()

# Print the results
print('Column Noisiness:')
print('-------------------')
print('Missing values:')
print(missing_perc)
print('Unique values:')
print(unique_perc)
print('-------------------')
print('Type of noise:')
print('-------------------')
print('Skewness:')
print(skewness)

## Unwanted entries and recording errors

In [ ]:
df_X.iloc[:, :60].plot(lw=0, marker=".", subplots=True, layout=(-1, 4), figsize=(15, 30), markersize=6);

## Non-numerical features

In [ ]:
# Extract descriptive properties of non-numerical features
df_X.describe(exclude=["number", "datetime"])

In [ ]:
# Create figure object with 3 subplots
fig, axes = plt.subplots(ncols=1, nrows=7, figsize=(12, 16))

# Identify non-numerical features
df_non_numerical = df_X.select_dtypes(exclude=["number", "datetime"])

# Loop through features and put each subplot on a matplotlib axis object
for col, ax in zip(df_non_numerical.columns, axes.ravel()):

    # Selects one single feature and counts number of occurrences per unique value
    df_non_numerical[col].value_counts().plot(

        # Plots this information in a figure with log-scaled y-axis
        logy=True, title=col, lw=0, marker=".", ax=ax)
    
plt.tight_layout();

# Content Investigation

## Feature Distribution

In [ ]:
df_X_50 = df_X.iloc[:, :55] # Select the first 55 columns
# Plots the histogram for each numerical feature in a separate subplot
df_X_50.hist(bins=25, figsize=(25, 45), layout=(-1, 5), edgecolor="black")
# Save the figure to the results folder as a PNG file
plt.savefig('results/feature_distribution.png', dpi=300, bbox_inches='tight')
plt.show()  

In [ ]:
# Collects for each feature the most frequent entry
most_frequent_entry = df_X_50.mode()

# Checks for each entry if it contains the most frequent entry
df_freq_50 = df_X_50.eq(most_frequent_entry.values, axis=1)

# Computes the mean of the 'is_most_frequent' occurrence
df_freq_50 = df_freq_50.mean().sort_values(ascending=False)

# Show the 5 top features with the highest ratio of singular value content
display(df_freq_50.head())

# Visualize the 'df_freq' table
df_freq_50.plot.bar(figsize=(15, 4));

In [ ]:
import seaborn as sns

# Select the numeric columns
numeric_cols = df.iloc[:, :61].select_dtypes(include=['float64', 'int64']).columns.tolist()

# Plot the histograms of patients without Neuro event
for col in numeric_cols:
    elected_rows = df[df['NP-SLE'] == 0]
    fig, ax = plt.subplots()
    sns.histplot(data=df, x=col, kde=True, ax=ax)
    ax.axvline(df[col].mean(), color='red', linestyle='--', label='Mean')
    ax.axvline(df[col].median(), color='green', linestyle='--', label='Median')
    ax.axvline(df[col].std(), color='orange', linestyle='--', label='Standard Deviation')
    ax.set_title(f'Distribution of {col}')
    ax.legend()
    # Save the figure to the results folder as a PNG file
    fig.savefig(f'results/Neuro_2/{col}_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()  #
    plt.close(fig)

In [ ]:
# Plot the histograms of patients with Neuro event
for col in numeric_cols:
    elected_rows = df[df['NP-SLE'] == 1]
    fig, ax = plt.subplots()
    sns.histplot(data=df, x=col, kde=True, ax=ax)
    ax.axvline(df[col].mean(), color='red', linestyle='--', label='Mean')
    ax.axvline(df[col].median(), color='green', linestyle='--', label='Median')
    ax.axvline(df[col].std(), color='orange', linestyle='--', label='Standard Deviation')
    ax.set_title(f'Distribution of {col}')
    ax.legend()
    # Save the figure to the results folder as a PNG file
    fig.savefig(f'results/Neuro_2/{col}_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()  
    plt.close(fig)

In [ ]:
# Plot the boxplots with quartiles and median of patients without Neuro event
for col in numeric_cols:
    selected_rows = df[df['NP-SLE'] == 0]
    fig, ax = plt.subplots()
    sns.boxplot(data=selected_rows, x=col, showfliers=False)
    ax.set_title(f'Distribution of {col}')
    # Add median line
    median = selected_rows[col].median()
    ax.axvline(median, color='red', label=f'Median: {median:.2f}')
    # Add quartile lines
    q1, q3 = selected_rows[col].quantile(q=[0.25, 0.75])
    ax.axvline(q1, color='green', label=f'Q1: {q1:.2f}')
    ax.axvline(q3, color='orange', label=f'Q3: {q3:.2f}')
    ax.legend()
    # Save the figure to the results folder as a PNG file
    fig.savefig(f'results/Neuro_2/{col}_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()  
    plt.close(fig)

In [ ]:
# Plot the boxplots with quartiles and median of patients without Neuro event
for col in numeric_cols:
    selected_rows = df[df['NP-SLE'] == 1]
    fig, ax = plt.subplots()
    sns.boxplot(data=selected_rows, x=col, showfliers=False)
    ax.set_title(f'Distribution of {col}')
    # Add median line
    median = selected_rows[col].median()
    ax.axvline(median, color='red', label=f'Median: {median:.2f}')
    # Add quartile lines
    q1, q3 = selected_rows[col].quantile(q=[0.25, 0.75])
    ax.axvline(q1, color='green', label=f'Q1: {q1:.2f}')
    ax.axvline(q3, color='orange', label=f'Q3: {q3:.2f}')
    ax.legend()
    # Save the figure to the results folder as a PNG file
    fig.savefig(f'results/Neuro_2/{col}_distribution.png', dpi=300, bbox_inches='tight')
    plt.show()  
    plt.close(fig)

In [ ]:
import numpy as np
from scipy import stats

#df = df.drop(['Patient','Date of Birth', 'Gender', 'Education', 'Age'], axis = 'columns')

# Select a numeric column for demonstration
numeric_col = 'Brain (WM+GM) volume cm3'

# Generate the theoretical quantiles for a normal distribution
normal_theoretical = np.random.normal(loc=np.mean(df[numeric_col]), scale=np.std(df[numeric_col]), size=len(df[numeric_col]))

# Calculate the sample quantiles
sample_quantiles = np.sort(df[numeric_col])

# Create the QQ plot
stats.probplot(df[numeric_col], dist='norm', plot=plt)

# Show the plot
plt.title('QQ Plot of {}'.format(numeric_col))
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Sample Quantiles')
# Save the figure to the results folder as a PNG file, before showing the plot
plt.savefig('results/QQ_Abnormal_white.png', dpi=300, bbox_inches='tight')
# Show the plot
plt.show()

In [ ]:
import statsmodels.api as sm

# Select variables with over 10 unique values
unique_counts = df.nunique()
vars_to_plot = df.iloc[:, :51]
vars_to_plot = unique_counts[unique_counts > 10].index.tolist()

# Loop over variables and create QQ plots
for var in vars_to_plot:
    # Calculate mean and standard deviation of the variable
    mean = df[var].mean()
    std = df[var].std()
    
    # Generate theoretical quantiles based on the mean and standard deviation
    theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(df[var])))
    
    # Create QQ plot
    fig = sm.qqplot(df[var], line='s')
    plt.plot(theoretical_quantiles, df[var].sort_values(), 'o', color='gray', alpha=0.5)
    plt.title(f'QQ Plot for {var}')
    plt.show();

## Feature Patterns

In [ ]:
df_X[["Prednisone  (mg equivalent)", "C3 (mg/dl)",
      "SLEDAI-2k (at the time of NP event)", "anti-dsDNA Titre  (insert NV here <7 )"]].plot(
    lw=0, marker=".", subplots=True, layout=(-1, 2), markersize=5, figsize=(15, 6));

In [79]:
# Creates mask to identify numerical features with more or less than 5 unique features
col_continuous = df_X.select_dtypes(include="number").nunique() >= 5

## Continuous Feature

In [80]:
# Create a new dataframe which only contains the continuous features
df_continuous = df_X[col_continuous[col_continuous].index]
df_continuous.shape

(27, 936)

In [ ]:
sns.set(font_scale=2)
sns.pairplot(df_continuous.iloc[:,:10],height=5, plot_kws={"s": 50, "alpha": 1});

In [ ]:
sns.set(font_scale=2)
sns.pairplot(df_continuous.iloc[:,10:21], height=5, plot_kws={"s": 50, "alpha": 1});

In [ ]:
sns.pairplot(
    df_X, plot_kws={"s": 10, "alpha": 1}, hue="NP-SLE", palette=sns.color_palette('bright', df_X["NP-SLE"].unique().shape[0]),
    x_vars=["Prednisone  (mg equivalent)", "anti-dsDNA Titre  (insert NV here <7 )", "C3 (mgdl)"],
    y_vars="SLEDAI-2k (at the time of NP event)");
# Save the figure to the results folder as a PNG file
plt.savefig('results/pair_sledai_prednisnone_dsDNa_c3.png', dpi=300, bbox_inches='tight')

## Discrete and Ordinal Features

In [81]:
# Create a new dataframe which doesn't contain the numerical continuous features
df_discrete = df_X[col_continuous[~col_continuous].index]
df_discrete.shape

(27, 25)

In [ ]:
import numpy as np

# Establish number of columns and rows needed to plot all features
n_cols = 5
n_elements = len(df_discrete.columns)
n_rows = np.ceil(n_elements / n_cols).astype("int")

# Specify y_value to spread data (ideally a continuous feature)
y_value = df_X["Disease duration (months)"]

# Create figure object with as many rows and columns as needed
fig, axes = plt.subplots(ncols=n_cols, nrows=n_rows, figsize=(15, n_rows * 2.5))

# Loop through features and put each subplot on a matplotlib axis object
for col, ax in zip(df_discrete.columns, axes.ravel()):
    sns.stripplot(data=df_X, x=col, y=y_value, ax=ax, palette="tab10", size=10, alpha=0.5)
plt.tight_layout();
# Save the figure to the results folder as a PNG file
plt.savefig('results/stripplot_disease_dur_vs_continuous.png', dpi=300, bbox_inches='tight')

In [ ]:
# Specify features of interest
selected_features = ["anti-dsDNA Titre  (0=absent; 1=present) )", "aPL syndrome",
                     "AnAb ", "Anti-Rib-P",
                     "NP-SLE", "Anti-DWEYS"]

# Create a figure with 3 x 2 subplots
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(16, 8))

# Loop through these features and plot entries from each feature against `Disease_duration`
for col, ax in zip(selected_features, axes.ravel()):
    sns.stripplot(data=df_X, x=col, y=df_X["Disease duration (months)"], ax=ax,
                  palette="tab10", size=10, alpha=0.5)
plt.tight_layout();
# Save the figure to the results folder as a PNG file
plt.savefig('results/stripplot_disease_dur_vs_features.png', dpi=300, bbox_inches='tight')

In [ ]:
# Create a figure with 3 x 2 subplots
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(16, 8))
new_selected_features = ["anti-dsDNA Titre  (0=absent; 1=present) )", "aPL syndrome",
                     "AnAb ", "Anti-Rib-P",
                     "SLICC-DI (at the time of NP event)", "Anti-DWEYS"]
# Loop through these features and plot entries from each feature against `Disease_Duration`
for idx, col in enumerate(new_selected_features):
    ax = axes[idx // 3, idx % 3]
    sns.violinplot(data=df_X, x=col, y=df_X["Disease duration (months)"], palette="Set2", split=True, hue="NP-SLE", ax=ax)
plt.tight_layout()
# Save the figure to the results folder as a PNG file
plt.savefig('results/violin_disease_against_features.png', dpi=300, bbox_inches='tight')

## Feature Relationships

In [85]:
# Computes feature correlation
df_corr = df_X.corr(method="pearson")

/var/folders/gv/8pp24fhj55x0_14187vq2lkh0000gp/T/ipykernel_2643/1279958852.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_corr = df_X.corr(method="pearson")


In [ ]:
# Create labels for the correlation matrix
labels = np.where(np.abs(df_corr)>0.75, "S",
                  np.where(np.abs(df_corr)>0.5, "M",
                           np.where(np.abs(df_corr)>0.25, "W", "")))

# Plot correlation matrix
plt.figure(figsize=(15, 15))
sns.heatmap(df_corr, mask=np.eye(len(df_corr)), square=True,
            center=0, annot=labels, fmt='', linewidths=.5,
            cmap="vlag", cbar_kws={"shrink": 0.8})
# Save the figure to the results folder as a PNG file
plt.savefig('results/correlation_matrix.png', dpi=300, bbox_inches='tight')

In [ ]:
# Sort by correlation with target column
target_column = 'NP-SLE'
sorted_corr = df_corr[target_column].sort_values()

# Plot sorted correlation matrix
sns.heatmap(df_corr.loc[sorted_corr.index, sorted_corr.index], cmap="YlGnBu")
# Save the figure to the results folder as a PNG file
plt.savefig('results/correlation_target_event.png', dpi=300, bbox_inches='tight')

In [ ]:
#  Creates a mask to remove the diagonal and the upper triangle.
lower_triangle_mask = np.tril(np.ones(df_corr.shape), k=-1).astype("bool")

#  Stack all correlations, after applying the mask
df_corr_stacked = df_corr.where(lower_triangle_mask).stack().sort_values()

#  Showing the lowest and highest correlations in the correlation matrix
display(df_corr_stacked)

In [ ]:
# Select the row of the correlation matrix corresponding to 'result'
pearson_corr = df_corr.loc['NP-SLE',:]

# Sort the correlations in descending order
pearson_corr = pearson_corr.sort_values(ascending=False)

# Select the top 10 correlated variables
top_10_corr = pearson_corr.iloc[1:11]

# Plot the top 10 correlations using a barplot
sns.set_style("whitegrid")
plt.figure(figsize=(30,10))
ax = sns.barplot(x=top_10_corr.index, y=top_10_corr.values)
plt.title('Top 10 MRI & Clinical variables correlated with np event')
plt.xlabel('MRI & Clinical variables')
plt.ylabel('Pearson correlation coefficient')
# Save the figure to the results folder as a PNG file
plt.savefig('results/10_correlated_features.png', dpi=300, bbox_inches='tight')
plt.show()